In [1]:
import numpy as np
import numpy.random as random

In [2]:
import importlib

In [241]:
np.reshape( [[1, 0, 0],[1,0,0], [1,0,0]], 9)
np.zeros([3,3])

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [247]:
I = 0
n=3
np.reshape([[1 if i==I else 0 for j in range(n)] for i in range(n)], n*n)

array([1, 1, 1, 0, 0, 0, 0, 0, 0])

In [280]:
class LinearBoard:
    def __init__(self, n=3):
        self.n = n
        self.board = np.zeros(n*n)
        self.initMasks()
        
    def initMasks(self):
        masks = []
        # rows
        for I in range(n):
            masks = masks + [np.reshape([[1 if i==I else 0 for j in range(n)] for i in range(n)], n*n)]
        # columns
        for J in range(n):
            masks = masks + [np.reshape([[1 if j==J else 0 for j in range(n)] for i in range(n)], n*n)]
        # diagonals
        masks = masks + [np.reshape([[1 if i==j else 0 for j in range(n)] for i in range(n)], n*n)]
        masks = masks + [np.reshape([[1 if i==n-j else 0 for j in range(n)] for i in range(n)], n*n)]
        self.masks = np.array(masks)
        return masks
    
    def ij_to_ind(i, j):
        return i*self.n + j
    def ind_to_ij(ind):
        return [ind//self.n, i % self.n]
    
    def printBoard(self, printWinner=True):
        board = self.board.reshape([self.n, self.n])
        markers = dict({0:' ', 1:"X", 2:"\x1b[31m0\x1b[0m"})
        print("    ", end="")
        for j in range(len(board[0])):
            print(j, end=" ");
        print("")
        for i in range(len(board)):
            print(i," |", end="")
            for j in range(len(board[i])):
                  print(markers[board[i][j]], end=" ")
            print("|")
        # if printWinner:
        #     print("Winner:", self.getWinner())
        

        

In [279]:
(1 // 3, 1 % 3)

(0, 1)

In [273]:
b = LinearBoard(3)
b.board[0]=1
b.board[1]=2
b.board[2]=1
b.printBoard()

    0 1 2 
0  |X 0 X |
1  |      |
2  |      |


In [3]:
import board

In [73]:
importlib.reload(board);

In [74]:
B = board.Board(n=3)
B.movesToBoard([ (1,[0,0]), (1,[0,1]), (2,[0,2])])
B.printBoard()

    0 1 2 
0  |X X 0 |
1  |      |
2  |      |
Winner: -1


In [75]:
B2 = B.copy()

In [70]:
def copy(b):
    newB = board.Board()
    newB.board = b.board.copy()
    newB.masks = b.masks.copy()
    return newB

In [76]:
B2 = B.copy()
B2.board[1][0]=1
B2.printBoard()


    0 1 2 
0  |X X 0 |
1  |X     |
2  |      |
Winner: -1


In [77]:
B.printBoard()

    0 1 2 
0  |X X 0 |
1  |      |
2  |      |
Winner: -1


In [78]:
def simulateGame(B, p1=None, p2=None, rnd=0, playerToMove_ = 1, max_moves = 10000):
    history = B.boardToMoves()
    board = B.board
    masks = B.masks
    n_moves = 0
    playerToMove = playerToMove_
    while B.getWinner() == -1 and n_moves<max_moves:
        n_moves = n_moves+1
        # Chose a move (random or use a player model if provided)
        move = None
        moves = B.getMoves()
        move = moves[random.randint(0, len(moves) )]
       
        # Make the move
        B.board[move[0]][move[1]] = playerToMove
        
        # Add the move to the history
        history.append((playerToMove, move))
        
        # Switch the active player
        playerToMove = 1 if playerToMove == 2 else 2
        
    return history

In [230]:
B = board.Board(5)
simulateGame(B, max_moves=1);
B.printBoard()

    0 1 2 3 4 
0  |          |
1  |X         |
2  |          |
3  |          |
4  |          |
Winner: -1


In [236]:
moves = B.getMoves()
probe_MC = np.zeros( len(moves))
for iMove in range(len(moves)):
    m = moves[iMove]
    probe_results = np.empty([], dtype = 'int')
    for _ in range(100):
        B2 = B.copy()
        B2.board[m[0]][m[1]] = 2
        simulateGame(B2, playerToMove_=1)
        probe_results = np.append(probe_results, B2.getWinner())
    probe_MC[iMove] = np.mean( probe_results == 2)

In [235]:
best_move = moves[np.argmax(probe_MC)]
best_move

[0, 4]

In [229]:
B.board[best_move[0]][best_move[1]] = 2;
B.printBoard()

    0 1 2 3 4 
0  |    0     |
1  |      X   |
2  |    0     |
3  |          |
4  |          |
Winner: -1
